In [ ]:
# Import necessary libraries
import re
import yaml
from typing import Any, Literal, NamedTuple

In [ ]:
# Define the Expr class with AND/OR support
class Expr:
    def __init__(self, args: str):
        self.args = args
        self.op = 'expr'  # To match the expected BaseOperator structure

    def test(self, o: dict[str, Any]) -> bool:
        """
        Evaluates expressions like '$.foo == "some value"' against the provided object.
        Also supports logical operators AND and OR for combining conditions:
        '$.foo == "value" AND $.count > 5'
        '$.status == "active" OR $.role == "admin"'

        The $ symbol represents the root of the object being tested.
        Supported comparison operators: ==, !=, >, <, >=, <=
        Supported logical operators: AND, OR (case insensitive)
        """
        import re

        # Split on AND/OR (but not inside quotes)
        def split_on_logical_ops(expr):
            parts = []
            operators = []
            last_pos = 0
            in_quotes = False

            for i, char in enumerate(expr):
                if char in ['"', "'"]:
                    in_quotes = not in_quotes

                if not in_quotes and i + 5 <= len(expr):
                    if expr[i:i+5].upper() == ' AND ':
                        parts.append(expr[last_pos:i])
                        operators.append('AND')
                        last_pos = i + 5
                    elif expr[i:i+4].upper() == ' OR ':
                        parts.append(expr[last_pos:i])
                        operators.append('OR')
                        last_pos = i + 4

            parts.append(expr[last_pos:])
            return parts, operators

        # Evaluate a single comparison expression
        def evaluate_comparison(expr):
            # Parse expression: $.fieldpath operator value
            pattern = r'\$\.([\w\.]+)\s*(==|!=|>|<|>=|<=)\s*(.+)'
            match = re.match(pattern, expr.strip())

            if not match:
                raise ValueError(f"Invalid expression format: {expr}")

            field, operator, value = match.groups()

            # Try to evaluate the value part (handles quoted strings, numbers, etc.)
            try:
                evaluated_value = eval(value, {"__builtins__": {}}, {})
            except:
                evaluated_value = value  # Keep as string if eval fails

            # Get the field value from the object
            try:
                import pydash
                field_value = pydash.get(o, field)
            except ImportError:
                # Fallback for getting nested fields
                parts = field.split('.')
                field_value = o
                for part in parts:
                    if isinstance(field_value, dict) and part in field_value:
                        field_value = field_value[part]
                    else:
                        field_value = None
                        break

            # Apply the operator
            if operator == "==":
                return field_value == evaluated_value
            elif operator == "!=":
                return field_value != evaluated_value
            elif operator == ">":
                return field_value > evaluated_value
            elif operator == "<":
                return field_value < evaluated_value
            elif operator == ">=":
                return field_value >= evaluated_value
            elif operator == "<=":
                return field_value <= evaluated_value
            else:
                raise ValueError(f"Unsupported operator: {operator}")

        # If there are no logical operators, evaluate as a simple expression
        if ' AND ' not in self.args.upper() and ' OR ' not in self.args.upper():
            return evaluate_comparison(self.args)

        # Split the expression by logical operators
        parts, operators = split_on_logical_ops(self.args)

        # Evaluate the first part
        result = evaluate_comparison(parts[0])

        # Apply each logical operator in sequence
        for i, op in enumerate(operators):
            if op == 'AND':
                result = result and evaluate_comparison(parts[i+1])
                # Short-circuit evaluation - if already False, no need to continue with AND
                if not result:
                    break
            elif op == 'OR':
                result = result or evaluate_comparison(parts[i+1])
                # Short-circuit evaluation - if already True, no need to continue with OR
                if result and (i+1 < len(operators) and operators[i+1] == 'OR'):
                    continue

        return result

In [ ]:
# Create test objects for evaluation
test_objects = {
    "simple": {
        "name": "John Doe",
        "age": 30,
        "active": True,
        "tags": ["developer", "python"]
    },
    "nested": {
        "user": {
            "profile": {
                "name": "Jane Smith",
                "age": 25,
                "preferences": {
                    "theme": "dark",
                    "notifications": True
                }
            },
            "permissions": ["read", "write", "admin"]
        },
        "metadata": {
            "created": "2023-01-15",
            "modified": "2023-05-20"
        }
    },
    "array_test": {
        "items": [
            {"id": 1, "value": "first"},
            {"id": 2, "value": "second"}
        ],
        "counts": [5, 10, 15, 20],
        "flags": [True, False, True]
    }
}

In [ ]:
# Basic comparison tests
print("BASIC COMPARISON TESTS")
print("----------------------")

basic_tests = [
    ("$.name == 'John Doe'", test_objects["simple"]),
    ("$.age > 25", test_objects["simple"]),
    ("$.active == True", test_objects["simple"]),
    ("$.user.profile.name == 'Jane Smith'", test_objects["nested"]),
    ("$.user.profile.age < 30", test_objects["nested"]),
    ("$.user.profile.preferences.theme == 'dark'", test_objects["nested"]),
    ("$.metadata.created != 'unknown'", test_objects["nested"]),
    ("$.counts[0] == 5", test_objects["array_test"]),
]

for expr_str, obj in basic_tests:
    expr = Expr(expr_str)
    result = expr.test(obj)
    print(f"Expression: {expr_str}")
    print(f"Result: {result}")
    print()

In [ ]:
# AND operator tests
print("\nAND OPERATOR TESTS")
print("----------------")

and_tests = [
    ("$.name == 'John Doe' AND $.age > 25", test_objects["simple"]),  # True AND True = True
    ("$.name == 'John Doe' AND $.age < 25", test_objects["simple"]),  # True AND False = False
    ("$.age > 20 AND $.active == True AND $.tags[0] == 'developer'", test_objects["simple"]),  # Multiple ANDs
    ("$.user.profile.age < 30 AND $.user.permissions[2] == 'admin'", test_objects["nested"]),  # Nested properties
]

for expr_str, obj in and_tests:
    expr = Expr(expr_str)
    result = expr.test(obj)
    print(f"Expression: {expr_str}")
    print(f"Result: {result}")
    print()

In [ ]:
# OR operator tests
print("\nOR OPERATOR TESTS")
print("---------------")

or_tests = [
    ("$.name == 'John Smith' OR $.age > 25", test_objects["simple"]),  # False OR True = True
    ("$.name == 'John Smith' OR $.age < 25", test_objects["simple"]),  # False OR False = False
    ("$.age < 20 OR $.active == False OR $.tags[0] == 'developer'", test_objects["simple"]),  # Multiple ORs
    ("$.user.profile.age > 30 OR $.user.permissions[2] == 'admin'", test_objects["nested"]),  # Nested properties
]

for expr_str, obj in or_tests:
    expr = Expr(expr_str)
    result = expr.test(